## Workspace setup

In [ ]:
from datetime import datetime  
import uproot
from functools import partial
import numpy as np
import tensorflow as tf
import importlib

## Training dataset preparation

In [ ]:
import io_functions as io
importlib.reload(io)

import plotting_functions as plf
importlib.reload(plf)

batchSize = 18

#dataPath = '/scratch_hdd/akalinow/ELITPC/data/PythonAnalysis/E_11_sigma_2/'
#test_files = [dataPath+'out_C_arr_8.root:TPCData']
dataPath = '/scratch_cmsse/akalinow/ELITPC/data/HIgS_2022/20220412_extTrg_CO2_190mbar_DT1470ET/11.5MeV/EventTPC/2022-04-12T08-03-44/'
test_files = [dataPath+'EventTPC_2022-04-12T08:03:44.531_0000.root:TPCData']

datasetGenerator = partial(io.generator, files=test_files, batchSize=batchSize, features_only=True)

test_dataset = tf.data.Dataset.from_generator(
     datasetGenerator,
     output_signature=(
         tf.TensorSpec(shape=(batchSize,) + (io.projections.shape), dtype=tf.float32),
          tf.TensorSpec(shape=(batchSize,9), dtype=tf.float32)))

In [ ]:
for aBatch in test_dataset.take(5):
    plf.plotEvent(aBatch, model=None)
    pass

## Model performance on training data.

Fill Pandas DataFrame with true and response values.

In [ ]:
%%time
import utility_functions as utils
importlib.reload(utils)

model_path = "./training/0020_2023_Jun_02_13_37_57"

model = tf.keras.models.load_model(model_path)

df = utils.df

for aBatch in test_dataset: 
    df = utils.fillPandasDataset(aBatch, df, model)     
    
for aBatch in test_dataset.take(5):
    plf.plotEvent(aBatch, model=model)

df.describe()    

### Resolution plots

In [ ]:
import plotting_functions as plf
importlib.reload(plf)

#plf.controlPlots(df)
plf.plotEndPointRes(df=df, edge="Start", partIdx=1)
plf.plotEndPointRes(df=df, edge="Stop", partIdx=1)

plf.plotEndPointRes(df=df, edge="Start", partIdx=2)
plf.plotEndPointRes(df=df, edge="Stop", partIdx=2)

plf.plotLengthPull(df, partIdx=1)
plf.plotLengthPull(df, partIdx=2)
plf.plotLengthPullEvolution(df)
plf.plotOpeningAngleCos(df)

In [ ]:
mask = np.abs(df["GEN_StartPosX"] - df["RECO_StartPosX"])<3
df_filtered = df[mask]

mask = np.abs(df_filtered["GEN_StartPosY"] - df_filtered["RECO_StartPosY"])<3
df_filtered = df_filtered[mask]

mask = np.abs(df_filtered["GEN_StopPosX_Part1"] - df_filtered["RECO_StopPosX_Part1"])<10
df_filtered = df_filtered[mask]

print(df_filtered.describe())

plf.plotEndPointRes(df=df_filtered, edge="Start", partIdx=1)
plf.plotEndPointRes(df=df_filtered, edge="Stop", partIdx=1)

plf.plotEndPointRes(df=df_filtered, edge="Start", partIdx=2)
plf.plotEndPointRes(df=df_filtered, edge="Stop", partIdx=2)

plf.plotLengthPull(df_filtered, partIdx=1)
plf.plotLengthPull(df_filtered, partIdx=2)
plf.plotLengthPullEvolution(df_filtered)
plf.plotOpeningAngleCos(df_filtered)